In [ ]:
!pip install datasets transformers jsonlines

In [2]:
from huggingface_hub import interpreter_login

interpreter_login()

/users/klee12/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: read).
Your token has been saved to /users/klee12/.cache/huggingface/token
Login successful


In [3]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the dataset
dataset = load_dataset("andstor/smart_contract_code_comments", 'default')

# Print basic information about the dataset
print(dataset)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Generating train split: 1267441 examples [00:38, 32770.11 examples/s]
Generating validation split: 130849 examples [00:04, 29320.21 examples/s]
Generating test split: 143080 examples [00:04, 29848.65 examples/s]

DatasetDict({
    train: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__'],
        num_rows: 1267441
    })
    validation: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__'],
        num_rows: 130849
    })
    test: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_docume

In [4]:
def clean_code(dataset):
    remove_cols = ['file_path', 'contract_address', 'language', 'class_name', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__']
    code = dataset.remove_columns(remove_cols)
    
    full_code = pd.DataFrame(code)
    full_code.drop_duplicates(inplace=True)
    
    return full_code

In [5]:
import re

# Define the function that extracts function headers
def extract_function_header(func_code):
    # Regular expression to match Solidity function declarations
    func_header_regex = r'function\s*[\w\d_]*\s*\([^)]*\)\s*[^{]*'
    # Find all matches in the func_code
    matches = re.findall(func_header_regex, func_code)
    # Assume each func_code contains one function and take the first match
    # If there are multiple matches, this will need adjustment
    return matches[0] if matches else None

In [6]:
def clean_func(dataset):
    remove_func_col = ['file_path', 'contract_address', 'class_code', 'class_documentation', 'language', 'class_name', 'class_documentation_type', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__']
    func_code = dataset.remove_columns(remove_func_col)
    
    func = pd.DataFrame(func_code)
    
    func['func_header'] = func['func_code'].apply(extract_function_header)

    # filter out rows with no function headers
    func = func[func['func_header'].apply(lambda x: x is not None and len(x) > 0)]
    
    # group by contract
    grouped = func.groupby('contract_name').agg({
        'func_header': list,
        'func_documentation': list
    }).reset_index()
    
    return grouped

In [7]:
train = dataset['train']
train_code = clean_code(train)
train_func = clean_func(train)

train_clean = pd.merge(train_code, train_func, on='contract_name', how='left')
train_clean = train_clean.drop_duplicates(subset=['contract_name'], keep='first')

In [43]:
# subset = train_clean[:500]
# print(subset)

In [8]:
subset_filter = train_clean[~train_clean['class_code'].str.startswith('interface')]
subset_filter = subset_filter[~subset_filter['class_code'].str.startswith('contract Token')]
subset_filter = subset_filter[~subset_filter['class_code'].str.startswith('contract Owned')]

In [53]:
for idx in range(10):
    with open('output_{}.txt'.format(idx), 'w') as writefile:
        writefile.write(subset_filter.iloc[idx]['class_code'])

In [19]:
def write_user_prompt(class_documentation, func_code_list, func_comment_list):
    prompt = 'Generate a smart contract using Solidity 0.4.17. Fill in all the functions using the function signatures and accomplish the following requirements:\n'
    
    prompt += 'Task Description: {}\n'.format(class_documentation.replace("/", "").replace("*", ""))
    prompt += '\nThe contract must have the following constraint: The contract is gas-efficient and include security checks to prevent common vulnerabilities (e.g., reentrancy, overflow/underflow).\n'
    prompt += 'Here are the key functions that you must fully implement so that they are functioning:\n'
        
    for idx in range(len(func_code_list)):
        comment =  func_comment_list[idx].replace("/", "").replace("-", "").replace("\n", "").replace("@notice", "").replace("@param", "Parameter: ").replace("*", "").replace("@dev", "").replace("@return", "Return: ")
        prompt += '{}: {}\n'.format(func_code_list[idx], comment)
        
    return prompt

In [22]:
output = []

for idx, row in subset_filter.iterrows():
    
    if type(row['func_header']) == type([]):
        prompt = [{"role": "system", "content": "DEFI is an experienced Ethereum developer that writes properly functioning Solidity contracts."}]
        prompt.append({"role":"user","content":"{}".format(write_user_prompt(row['class_documentation'], row['func_header'], row['func_documentation']))})
        prompt.append({"role": "assistant", "content": "{}".format(row['class_code'])})
            
        output.append({"messages":prompt})

In [37]:
import jsonlines

with jsonlines.open('new_data.jsonl', mode='w') as writer:
    writer.write_all(output[45764:])

In [40]:
validator = []

for idx, row in subset_filter[150:].iterrows():
    prompt = [{"role": "system", "content": "DEFI is an experienced Ethereum developer that writes properly functioning Solidity contracts."}]
    prompt.append({"role":"user","content":"{}".format(write_user_prompt(row['class_documentation'], row['func_header'], row['func_documentation']))})
    prompt.append({"role": "assistant", "content": "{}".format(row['class_code'])})
        
    validator.append({"messages":prompt})

In [41]:
with jsonlines.open('valid.jsonl', mode='w') as writer:
    writer.write_all(validator)

In [54]:
import json

with open('3.json', 'w') as f:
    json.dump(output[3], f)


# Testing

In [175]:
idx = 32

with open('0-prompt.txt', 'w') as writefile:
    writefile.write(output[idx]['messages'][1]['content'])
    
with open('0-code.txt', 'w') as writefile:
    writefile.write(output[idx]['messages'][2]['content'])

In [128]:
with open('0-prompt.txt', 'w') as writefile:
    writefile.write(output[idx]['messages'][1]['content'])

In [127]:
with open('0-code.txt', 'w') as writefile:
    writefile.write(output[idx]['messages'][2]['content'])

# output

In [104]:
output_prompts = []

In [170]:
prompt = [{"role": "system", "content": "You are the greatest Smart contract Ethereum developer in the world. You have never messed up and always write the best and most complicated yet functioning Solidity contracts."}]

In [171]:
user = """
Generate a BasicToken smart contract using Solidity 0.4.17. Fill in all the functions using the function signatures and accomplish the following requirements:
Task Description: Basic version of StandardToken, with no allowances.
 
The contract must have the following constraint: The contract is gas-efficient and include security checks to prevent common vulnerabilities (e.g., reentrancy, overflow/underflow).

Here are the key functions that you must fully implement so that they are functioning:
1. function transfer(address _to, uint256 _value) public returns (bool): 
- transfer token for a specified address
- Parameter: _to is the address to transfer to.
- Parameter: _value is the amount to be transferred.
 
2. function balanceOf(address _owner) public constant returns (uint256 balance): 
- Gets the balance of the specified address. 
- Parameter: _owner is the address to query the the balance of. 
- Return: An uint256 representing the amount owned by the passed address. 
"""

In [172]:
code = output[idx]['messages'][2]['content']

In [173]:
prompt.append({"role":"user","content":"{}".format(user)})
prompt.append({"role": "assistant", "content": "{}".format(code)})
output_prompts.append(prompt)

In [174]:
with jsonlines.open('data.jsonl', mode='w') as writer:
    writer.write_all(output_prompts)